In [2]:
import os
import json
import numpy as np
import pickle 
import tqdm

# Roles, currently doesn't distinguish between carry/support in safelane
# 1 safelane
# 2 mid
# 3 offlane
# 4 jungle
# 5 <- label by looking at wards placed and gold earned


"""
Features:
GPM: rank 1-5
XPM: rank 1-5
K+A: rank 1-5
D  : rank 1-5
Last-hits: rank 1-5
Team HIDS: Binary encoded [130]


Assumption: Linear difference between ranks
Goal: Create ensemble of classifiers where 
            |C| = #heroes * #roles

"""


if __name__ == "__main__":
    X = []
    y = []
    for filename in tqdm.tqdm(os.listdir("../data/parsed")):       
        with open("../data/parsed/" + filename, "rb") as f:
            match = pickle.load(f)
            players = match["players"]
            t0 = list(filter(lambda p: p["isRadiant"], players))
            t1 = list(filter(lambda p: not p["isRadiant"], players))
            t0_gpm_rank = sorted([(p["hero_id"], p["gold_per_min"]) for p in t0], key=lambda x: x[1], reverse=False)
            t1_gpm_rank = sorted([(p["hero_id"], p["gold_per_min"]) for p in t1], key=lambda x: x[1], reverse=False)
            t0_xpm_rank = sorted([(p["hero_id"], p["xp_per_min"]) for p in t0], key=lambda x: x[1], reverse=False)
            t1_xpm_rank = sorted([(p["hero_id"], p["xp_per_min"]) for p in t1], key=lambda x: x[1], reverse=False)
            t0_ka_rank = sorted([(p["hero_id"], p["kills"] + p["assists"]) for p in t0], key=lambda x: x[1], reverse=False)
            t1_ka_rank = sorted([(p["hero_id"], p["kills"] + p["assists"]) for p in t1], key=lambda x: x[1], reverse=False)
            t0_d_rank = sorted([(p["hero_id"], p["deaths"]) for p in t0], key=lambda x: x[1], reverse=False)
            t1_d_rank = sorted([(p["hero_id"], p["deaths"]) for p in t1], key=lambda x: x[1], reverse=False)
            t0_lasthits_rank = sorted([(p["hero_id"], p["last_hits"]) for p in t0], key=lambda x: x[1], reverse=False)
            t1_lasthits_rank = sorted([(p["hero_id"], p["last_hits"]) for p in t1], key=lambda x: x[1], reverse=False)
            t0_wards_placed = []
            t1_wards_placed = []

            for p in t0:
                item_uses = p["item_uses"]
                c = 0
                if "ward_observer" in item_uses:
                    c += item_uses["ward_observer"]
                if "ward_sentry" in item_uses:
                    c += item_uses["ward_sentry"]

                t0_wards_placed.append((p["hero_id"], c))

            for p in t1:
                item_uses = p["item_uses"]
                c = 0
                if "ward_observer" in item_uses:
                    c += item_uses["ward_observer"]
                if "ward_sentry" in item_uses:
                    c += item_uses["ward_sentry"]

                t1_wards_placed.append((p["hero_id"], c))

            print("=" * 10 + " t0_gpm_rank " + "=" * 10)
            print(t0_gpm_rank)

            print("=" * 10 + " t0_xpm_rank " + "=" * 10)
            print(t0_xpm_rank)

            print("=" * 10 + " t0_ka_rank " + "=" * 10)
            print(t0_ka_rank)

            print("=" * 10 + " t0_d_rank " + "=" * 10)
            print(t0_d_rank)

            print("=" * 10 + " t0_lasthits_rank " + "=" * 10)
            print(t0_lasthits_rank)

            print("=" * 10 + " t0_wards_placed " + "=" * 10)
            print(t0_wards_placed)
            print(t1_wards_placed)


            break
        
    # print(np.array(X).shape)
    # print(np.array(y).shape)

  0%|          | 0/50000 [00:00<?, ?it/s]

========== t0_gpm_rank ==========
[(101, 286), (135, 321), (97, 328), (59, 406), (44, 670)]
========== t0_xpm_rank ==========
[(101, 413), (97, 457), (59, 588), (135, 655), (44, 864)]
========== t0_ka_rank ==========
[(97, 13), (59, 16), (44, 17), (101, 21), (135, 23)]
========== t0_d_rank ==========
[(44, 3), (135, 6), (97, 8), (59, 12), (101, 12)]
========== t0_lasthits_rank ==========
[(101, 22), (135, 59), (97, 122), (59, 148), (44, 320)]
========== t0_wards_placed ==========
[(59, 2), (44, 0), (135, 7), (97, 0), (101, 6)]
[(19, 0), (84, 11), (81, 0), (75, 6), (74, 2)]


In [2]:
import pandas as pd

df_pos = pd.DataFrame(y)

df_pos[0].value_counts()


1    194482
3    192901
2    108191
4      4426
Name: 0, dtype: int64

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score,cross_val_predict, train_test_split
from sklearn.metrics import confusion_matrix


clf = DecisionTreeClassifier(random_state=0)
cross_val_score(clf, X, y, cv=10)

array([0.69526, 0.69326, 0.69306, 0.69764, 0.6958 , 0.69536, 0.69296,
       0.69408, 0.6944 , 0.69162])

In [4]:
y_pred = cross_val_predict(clf, X, y, cv=10)
confusion_matrix(y, y_pred)

array([[143465,  10368,  40649,      0],
       [ 16253,  67496,  24442,      0],
       [ 39045,  17645, 136211,      0],
       [  1951,    467,   2008,      0]])

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,cross_val_predict, train_test_split
from sklearn.metrics import confusion_matrix

clf = LogisticRegression(random_state=0)
y_pred = cross_val_predict(clf, X, y, cv=10)
confusion_matrix(y, y_pred)


array([[107285,      0,  87197,      0],
       [ 64956,      0,  43235,      0],
       [ 89258,      0, 103643,      0],
       [  2773,      0,   1653,      0]])

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.